## DOWNLOADING DATA FROM YAHOO FINANCE

In this file, we will firstly download stocks that are included in S&P 500 market index. Afterwards, we will use this to download a large dataset that contains financial data from Yahoo Finance (https://finance.yahoo.com/) for stocks in S&P 500. \\

In web app, we have chosen to download all data for all tickers from S&P 500 at once. At the beginning, the user will wait a while, however, if any changes are made, the user does not have to wait. 

In [2]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
from datetime import date
import urllib.request
import time
from bs4 import BeautifulSoup
import streamlit
import matplotlib.pyplot as plt
import yfinance as yf
import lxml
from pandas_datareader import DataReader
import yfinance as yf

## DOWNLOADING TICKERS FROM S&P 500 

Firstly, we get names for stock from S&P 500 market index. Then we get names of tickers. We also get the industry in which the company operates.

In [4]:
req = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(req.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

In [5]:
tickers=[]
sector = []
for row in table.findAll('tr')[1:]:
        tic = row.findAll('td')[0].text
        #sector
        sec = row.findAll('td')[4].text
        tickers.append(tic)
        sector.append(sec)

In [8]:
print(tickers)

['MMM\n', 'AOS\n', 'ABT\n', 'ABBV\n', 'ABMD\n', 'ACN\n', 'ATVI\n', 'ADM\n', 'ADBE\n', 'ADP\n', 'AAP\n', 'AES\n', 'AFL\n', 'A\n', 'AIG\n', 'APD\n', 'AKAM\n', 'ALK\n', 'ALB\n', 'ARE\n', 'ALGN\n', 'ALLE\n', 'LNT\n', 'ALL\n', 'GOOGL\n', 'GOOG\n', 'MO\n', 'AMZN\n', 'AMCR\n', 'AMD\n', 'AEE\n', 'AAL\n', 'AEP\n', 'AXP\n', 'AMT\n', 'AWK\n', 'AMP\n', 'ABC\n', 'AME\n', 'AMGN\n', 'APH\n', 'ADI\n', 'ANSS\n', 'ANTM\n', 'AON\n', 'APA\n', 'AAPL\n', 'AMAT\n', 'APTV\n', 'ANET\n', 'AJG\n', 'AIZ\n', 'T\n', 'ATO\n', 'ADSK\n', 'AZO\n', 'AVB\n', 'AVY\n', 'BKR\n', 'BLL\n', 'BAC\n', 'BBWI\n', 'BAX\n', 'BDX\n', 'BRK.B\n', 'BBY\n', 'BIO\n', 'TECH\n', 'BIIB\n', 'BLK\n', 'BK\n', 'BA\n', 'BKNG\n', 'BWA\n', 'BXP\n', 'BSX\n', 'BMY\n', 'AVGO\n', 'BR\n', 'BRO\n', 'BF.B\n', 'CHRW\n', 'CDNS\n', 'CZR\n', 'CPB\n', 'COF\n', 'CAH\n', 'KMX\n', 'CCL\n', 'CARR\n', 'CTLT\n', 'CAT\n', 'CBOE\n', 'CBRE\n', 'CDW\n', 'CE\n', 'CNC\n', 'CNP\n', 'CDAY\n', 'CERN\n', 'CF\n', 'CRL\n', 'SCHW\n', 'CHTR\n', 'CVX\n', 'CMG\n', 'CB\n', 'CHD\n', 

In [11]:
tickers = list(map(lambda s: s.strip(), tickers))

In [12]:
industries = list(map(lambda s: s.strip(), sector))

In [13]:
tickerdf = pd.DataFrame(tickers,columns=['ticker'])
sectordf = pd.DataFrame(industries,columns=['industry'])

In [14]:
df = pd.concat([tickerdf, sectordf], axis=1)
print(df)

    ticker                            industry
0      MMM            Industrial Conglomerates
1      AOS                   Building Products
2      ABT               Health Care Equipment
3     ABBV                     Pharmaceuticals
4     ABMD               Health Care Equipment
..     ...                                 ...
500    YUM                         Restaurants
501   ZBRA  Electronic Equipment & Instruments
502    ZBH               Health Care Equipment
503   ZION                      Regional Banks
504    ZTS                     Pharmaceuticals

[505 rows x 2 columns]


In [15]:
df2 = df.reindex(tickerdf.index)

In [16]:
df2

,ticker,industry
0,MMM,Industrial Conglomerates
1,AOS,Building Products
2,ABT,Health Care Equipment
3,ABBV,Pharmaceuticals
4,ABMD,Health Care Equipment
...,...,...
500,YUM,Restaurants
501,ZBRA,Electronic Equipment & Instruments
502,ZBH,Health Care Equipment
503,ZION,Regional Banks


So, tickers and industires are stored in dataframe. Now, we have to put tickers into respective format to be able to download financial data.

In [18]:
tick = df['ticker'].to_numpy()

In [19]:
tick_to_download = tick.tolist()
tick_to_download

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AAP',
 'AES',
 'AFL',
 'A',
 'AIG',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'CHRW',
 'CDNS',
 'CZR',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CERN',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMC

In [21]:
print(type(tick_to_download))

<class 'list'>


## Downloading Yahoo Finance data

Firstly,period for which data will be downloaded is set. Data will be downloaded from period starting on 1st January 2015 till today.

In [26]:
BEGINNING = "2015-01-01"
TODAY = date.today().strftime("%Y-%m-%d")

There are two approaches of downloading data. Firstly, data can be downloaded using yfinance package. The second approach is downloading data using DataReader packege which get data from various resources on Internet. 

# Yfinance approach

In [36]:
data_yf = yf.download(tick_to_download,start=BEGINNING,end=TODAY)

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


In [37]:
print(data_yf)

             Adj Close                                                 \
                     A        AAL         AAP        AAPL        ABBV   
Date                                                                    
2014-12-31   38.398003  50.814617  154.054565   25.057606   47.281742   
2015-01-02   38.134743  51.079918  153.358200   24.819241   47.606876   
2015-01-05   37.420185  51.051483  151.336792   24.120045   46.710945   
2015-01-06   36.837265  50.255585  151.230377   24.122320   46.479752   
2015-01-07   37.326168  50.227158  154.480148   24.460564   48.358299   
...                ...        ...         ...         ...         ...   
2022-01-28  137.059998  15.640000  228.220001  170.330002  137.919998   
2022-01-31  139.320007  16.469999  231.509995  174.779999  136.889999   
2022-02-01  141.029999  16.830000  234.800003  174.610001  137.000000   
2022-02-02  144.240005  16.959999  236.679993  175.839996  138.619995   
2022-02-03  141.789993  16.570000  232.490005  172.

In [38]:
data= data_yf.reset_index()
print(data)

           Date   Adj Close                                                 \
                          A        AAL         AAP        AAPL        ABBV   
0    2014-12-31   38.398003  50.814617  154.054565   25.057606   47.281742   
1    2015-01-02   38.134743  51.079918  153.358200   24.819241   47.606876   
2    2015-01-05   37.420185  51.051483  151.336792   24.120045   46.710945   
3    2015-01-06   36.837265  50.255585  151.230377   24.122320   46.479752   
4    2015-01-07   37.326168  50.227158  154.480148   24.460564   48.358299   
...         ...         ...        ...         ...         ...         ...   
1787 2022-01-28  137.059998  15.640000  228.220001  170.330002  137.919998   
1788 2022-01-31  139.320007  16.469999  231.509995  174.779999  136.889999   
1789 2022-02-01  141.029999  16.830000  234.800003  174.610001  137.000000   
1790 2022-02-02  144.240005  16.959999  236.679993  175.839996  138.619995   
1791 2022-02-03  141.789993  16.570000  232.490005  172.899994  

# DataReader approach

In Jupyter Notebook, it takes a very long time to run the function. Here is the function shown for a sample of 9 tickers.

In [48]:
def get_data_try():
    ticker_symbol=['AAPL','MSFT','GE','IBM','AA','DAL','UAL', 'PEP', 'KO']
    BEGINNING = "2015-01-01"
    TODAY = date.today().strftime("%Y-%m-%d")
    df1 = DataReader(ticker_symbol, 'yahoo', BEGINNING, TODAY)   
    df2=df1.reset_index()
    df2['Date']=df2['Date'].dt.date
    df3=df2.set_index('Date')
    df = pd.DataFrame(data=df3)
    return df

In [50]:
x=get_data_try()

In [53]:
x

Attributes   Adj Close                                                 \
Symbols           AAPL        MSFT          GE         IBM         AA   
Date                                                                    
2015-01-02   24.745995   41.193836  167.904007  114.906487  37.160915   
2015-01-05   24.048859   40.815025  164.821945  113.098442  35.008007   
2015-01-06   24.051125   40.215984  161.270935  110.659355  35.265423   
2015-01-07   24.388374   40.726929  161.337936  109.936180  36.178066   
2015-01-08   25.325432   41.925049  163.280930  112.325645  37.207706   
...                ...         ...         ...         ...        ...   
2022-01-31  174.779999  310.980011   94.480003  133.570007  56.709999   
2022-02-01  174.610001  308.760010   97.949997  135.529999  58.169998   
2022-02-02  175.839996  313.459991   98.040001  137.250000  59.209999   
2022-02-03  172.899994  301.250000   98.320000  137.779999  62.740002   
2022-02-04  172.445007  306.714996   98.675003  137.839996  64.950104   

Attributes                                                    Close  ...  \
Symbols           DAL        UAL         PEP         KO        AAPL  ...   
Date                                                                 ...   
2015-01-02  44.267319  66.339996   76.954910  33.559856   27.332500  ...   
2015-01-05  43.529240  66.150002   76.376350  33.559856   26.562500  ...   
2015-01-06  42.503120  64.580002   75.797798  33.814690   26.565001  ...   
2015-01-07  42.278091  65.529999   78.014198  34.236774   26.937500  ...   
2015-01-08  43.376209  66.639999   79.432076  34.650902   27.972500  ...   
...               ...        ...         ...        ...         ...  ...   
2022-01-31  39.689999  42.880001  173.520004  61.009998  174.779999  ...   
2022-02-01  40.490002  43.959999  172.339996  60.560001  174.610001  ...   
2022-02-02  40.520000  44.119999  175.470001  61.180000  175.839996  ...   
2022-02-03  39.730000  43.080002  175.369995  61.610001  172.899994  ...   
2022-02-04  39.450001  42.400002  173.169998  61.320000  172.445007  ...   

Attributes       Open       Volume                                    \
Symbols            KO         AAPL        MSFT         GE        IBM   
Date                                                                   
2015-01-02  42.259998  212818400.0  27913900.0  5319704.0  5779673.0   
2015-01-05  42.689999  257142000.0  39673900.0  5464316.0  5104898.0   
2015-01-06  42.410000  263188400.0  36447900.0  8288800.0  6429448.0   
2015-01-07  42.799999  160423600.0  29114100.0  5673525.0  4918083.0   
2015-01-08  43.180000  237458000.0  29645200.0  5619172.0  4431693.0   
...               ...          ...         ...        ...        ...   
2022-01-31  60.299999  115541600.0  46444500.0  7001600.0  5859000.0   
2022-02-01  60.910000   86213900.0  40950400.0  8149200.0  6206400.0   
2022-02-02  60.619999   84914300.0  36636000.0  5561400.0  5357200.0   
2022-02-03  60.939999   89254800.0  43660200.0  6212000.0  6100100.0   
2022-02-04  61.490002   40897499.0  16741070.0  2488398.0  1796778.0   

Attributes                                                            
Symbols            AA         DAL         UAL        PEP          KO  
Date                                                                  
2015-01-02  4340408.0   8637300.0   6215000.0  3545700.0   9921100.0  
2015-01-05  9026467.0  10556500.0   5033400.0  6441000.0  26292600.0  
2015-01-06  8063670.0  12880400.0   6051700.0  6195000.0  16897500.0  
2015-01-07  6637744.0  10516200.0   5135000.0  6526300.0  13412300.0  
2015-01-08  8185851.0  10499300.0   6889500.0  7131600.0  21743600.0  
...               ...         ...         ...        ...         ...  
2022-01-31  7206100.0  10657300.0  11871000.0  5908000.0  22045300.0  
2022-02-01  5939300.0   9668800.0   9614300.0  5952700.0  20841700.0  
2022-02-02  5613800.0  10249800.0  12135300.0  5767000.0  20225600.0  
2022-02-03  8070000.0   9911400.0   8003600.0  4632